<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [ ]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [ ]:
# LSST package for Butler queries
import lsst.daf.butler as dafButler

# LSST package for image display
import lsst.afw.display as afwDisplay

# Import general python packages
#import time
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal
import uuid
import requests
import warnings
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# Science Pipelines imports
from lsst.daf.butler import Butler, DatasetType, CollectionType
import lsst.geom as geom
import lsst.resources
from lsst.afw.image import Image, ImageF
from lsst.afw.image import Exposure, ExposureF

# Plotting with MPL
import matplotlib.pyplot as plt

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service

# PyVO packages
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Astropy
from astropy import units as u
#from astropy.coordinates import SkyCoord
#from astropy.io import fits
#from astropy.wcs import WCS       


### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

In [ ]:
def plotImage(exposure: ExposureF, img_opt: dict = None):
    """Plot and image using matplotlib
   
   Parameters
    ----------
    image : `Exposure`
        the image to plot
        
    opts : ``
   
   Returns
    -------
    title : `str` (only if result is not `None`)
        Plot title from string
    """
    
    fig, ax = plt.subplots()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(exposure.image)
    plt.show()


In [ ]:
def make_image_cutout(tap_service, ra, dec, cutout_size=0.01, 
                      imtype=None, dataId=None, filename=None): 
    """Make a cutout using the cutout tool
   
   Parameters
    ----------
    # cutout_size is in degrees
    # if imtype None, assumes deepCoadd. dataId must correspond to that for provided imtype
    # for deepCoadd, dataId should have format {'band':'band', 'tract':tract, 'patch':patch}
    # for calexp, dataId should have format {'visit':visitId, 'detector':detector}
   Returns
    -------
    
    """

    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    
    if imtype == 'calexp':
        
        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_visit, lsst_detector, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.calexp' " + \
            "AND lsst_visit = " + str(dataId["visit"]) + " " + \
            "AND lsst_detector = " + str(dataId["detector"])
        results = tap_service.search(query) # THIS IS TAP SERVICE

    else:
        # Find the tract and patch that contain this point
        # this is how its done using butler:
        tract = dataId["tract"]
        patch = dataId["patch"]
        
        # add optional default band if it is not contained in the dataId
        band = dataId["band"]
        print(band)

        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
            "AND lsst_tract = " + str(tract) + " " + \
            "AND lsst_patch = " + str(patch) + " " + \
            "AND lsst_band = " + "'" + str(band) + "' "
        results = tap_service.search(query) 

    #Get datalink
    dataLinkUrl = results[0].getdataurl()
    f"Datalink link service url: {dataLinkUrl}"
    auth_session = service._session
    dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
    f"{dl_results.status}"

    # from_resource: creates a instance from a number of records and a Datalink Resource.
    sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                                 session=auth_session)

    sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
                 spherePoint.getDec().asDegrees()*u.deg, 
                 cutout_size* u.deg)

    #sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/soda-cutout.fits')
    if filename:
        sodaCutout = 'cutouts/'+filename
        #sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/'+filename)
    else:
        #sodaCutout = 'cutouts/soda-cutout.fits'
        sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/soda-cutout.fits')

    with open(sodaCutout, 'bw') as f:
        f.write(sq.execute_stream().read())
        
    #sodaCutout is just the filename...
    return sodaCutout


## 2. Generating an image cutout using the cutout tool



In [ ]:
# Set afw display backend to matplotlib
afwDisplay.setDefaultBackend('matplotlib')

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry


In [ ]:
#service = get_tap_service()
service = get_tap_service("tap")

In [ ]:
# galaxy cluster:
#ra = 55.7467 
#dec = -32.2862

# vairable star from tutorial 07b
#ra_known_rrl  = 62.1479031
#dec_known_rrl  = -35.799138

# random DIAobject from a testing code (this ra/dec don't work for some reason
ra = 72.5383603
dec = -44.4248533

# Define a spatial point
spherePoint = lsst.geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)

#55.75067151347799 -32.27781914678513

# Find the tract and patch that contain this point
skymap = butler.get('skyMap') # skymap is used for coadd projection
tract = skymap.findTract(spherePoint)
patch = tract.findPatch(spherePoint)

# Now create a dataId from the tract, patch and filter information
dataId = {'band':'i', 'tract':tract.tract_id, 'patch': patch.getSequentialIndex()}

# And use the dataid to get the coadd
#deepCoadd = butler.get('deepCoadd_calexp', dataId=dataId)

#assert type(deepCoadd) == lsst.afw.image.ExposureF
#f"Tract: {tract.tract_id}, Patch: {patch.getSequentialIndex()}"



### 2.1 Test out a wrapper function "make_image_cutout" defined above, so we can do all the above operations in one go
IT WORKS :-D

In [ ]:
# Test our new function wrapper for the cutout service! IT WORKS :-D

imtype = 'deepCoadd' # this is the default, imtype=None will assume deepCoadd
test = make_image_cutout(service, ra, dec, dataId=dataId, imtype=imtype, cutout_size=0.005)
plotImage(ExposureF(test))


## 4. Creating cutouts of changing flux for the diaObject


In [ ]:
# maybe, instead, use Jeff's join query:
# do we need to sort out diaObjectId vs diaSourceId
diaObjectId = 1253478440036730088

ccdquery = "SELECT TOP 10 dia.coord_ra, dia.coord_dec, " + \
                        "dia.diaObjectId,  dia.ccdVisitId, dia.band, " + \
                        "cv.visitId, cv.physical_filter, cv.detector, cv.obsStartMJD, " + \
                         "cv.expMidptMJD " + \
                         "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject as dia " + \
                         "JOIN dp02_dc2_catalogs.CcdVisit as cv ON cv.ccdVisitId = dia.ccdVisitId " + \
                         "WHERE dia.diaObjectId = "+str(diaObjectId)+" AND cv.band = 'i'"
    
diaccdsearch = service.search(ccdquery)
forcedSrc = diaccdsearch.to_table()
forcedSrc
del diaccdsearch

In [ ]:
wh = np.argsort(forcedSrc['obsStartMJD'])
forcedSrc[wh]
#forcedSrc['detector'][wh]

In [ ]:
# This cell plots the cutouts by order of MJD 
#def make_image_cutout(butler, tap_service, ra, dec, cutout_size=0.01, imtype=None, visit=None, detector=None, filename=None):

for i in range(1):
    dataId_calexp = {'visit':forcedSrc['visitId'][wh][i], 'detector':forcedSrc['detector'][wh][i]}
    test = make_image_cutout(service, ra, dec, cutout_size=0.005,imtype='calexp', dataId=dataId_calexp,
                             filename='cutout_'+str(i)+'.fits')

    plotImage(ExposureF(test))


